# LangChain Api


## 初始化


## 初始化环境变量

In [ ]:
import { config } from "dotenv";

await config({ export: true });

const apiKey = Deno.env.get("API_KEY");
const baseURL = Deno.env.get("BASE_URL");
const systemPrompt = Deno.env.get("SYSTEM_PROMPT");


## 初始化 ai 实例

In [39]:
import { ChatOpenAI } from '@langchain/openai';

// 创建 ChatOpenAI 实例
const model = new ChatOpenAI({
    openAIApiKey: apiKey,
    configuration: {
      baseURL: baseURL,
    },
    modelName: "gpt-4o",
    temperature: 0.4,
    streaming: true
  });

## 通过LangChain 调用 AI Api


In [ ]:
import {  HumanMessage, SystemMessage } from '@langchain/core/messages';
import { ChatPromptTemplate } from '@langchain/core/prompts';
import { StringOutputParser } from '@langchain/core/output_parsers';
import { RunnableSequence } from '@langchain/core/runnables';

// 创建消息模板
const prompt = ChatPromptTemplate.fromMessages([
  new SystemMessage(systemPrompt),
  new HumanMessage("生成一个登陆界面")
]);

// LCEL 处理链写法1: pipe 写法
// const chain = prompt.pipe(model).pipe(new StringOutputParser());

// LCEL 处理链写法2: RunnableSequence
const chain = RunnableSequence.from([prompt, model, new StringOutputParser()]);

// 执行流式输出
const stream = await chain.stream();

let assistantContent = "";

// 处理流式响应
for await (const chunk of stream) {
  console.log(chunk);
  assistantContent += chunk;
}

console.log(assistantContent);

## 结构化输出



In [ ]:
import { StructuredOutputParser } from '@langchain/core/output_parsers';
import { ChatPromptTemplate } from '@langchain/core/prompts';
import { RunnableSequence } from '@langchain/core/runnables';
import { z } from 'zod';

const componentsResponseSchema = z.object({
  components: z.array(z.string()).describe("antd 组件列表")
});

const structuredOutputParser = StructuredOutputParser.fromZodSchema(componentsResponseSchema);

console.log(structuredOutputParser.getFormatInstructions());

const structurePrompt = ChatPromptTemplate.fromMessages([
  new SystemMessage(`提取出用户提供的代码中用到了哪些antd的基础组件 ${structuredOutputParser.getFormatInstructions()}`),
  new HumanMessage(assistantContent),
]);

const structureChain = RunnableSequence.from([structurePrompt, model, structuredOutputParser]);

const result = await structureChain.stream();

let components = "";
for await (const chunk of result) {
  console.log(chunk);
  components += chunk;
}

console.log(JSON.stringify(components));


> Langchain 更多资料详见官方文档：https://js.langchain.com/docs/tutorials/